In [8]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_probability as tfp


In [9]:
dim_z = 8

In [66]:
prior = tfp.distributions.Independent(
            tfp.distributions.Normal(loc=tf.zeros(dim_z), scale=1.0),
            reinterpreted_batch_ndims=1,
        )
input_img = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(
                filters=32,
                kernel_size=3,
                strides=(2, 2),
                padding="valid",
                activation="relu",
            ) (input_img)
x = layers.Conv2D(
                filters=64,
                kernel_size=3,
                strides=(2, 2),
                padding="valid",
                activation="relu",
            ) (x)
x = layers.Flatten()(x)
x = layers.Dense(
                tfp.layers.IndependentNormal.params_size(dim_z),
                activation=None,
                name="z_params",
            )(x)
encoded = tfp.layers.IndependentNormal(
                dim_z,
                activity_regularizer=tfp.layers.KLDivergenceRegularizer(
                    prior, weight=2.0
                ),
                name="z_layer",)(x)

decoder_input = layers.InputLayer(input_shape=dim_z)(encoded)
x = layers.Dense(7 * 7 * 32, activation=None)(decoder_input)
x = layers.Reshape((7, 7, 32))(x)
x = layers.Conv2DTranspose(
                filters=64, kernel_size=3, strides=2, padding="same", activation="relu"
            )(x)
x = layers.Conv2DTranspose(
                filters=32, kernel_size=3, strides=2, padding="same", activation="relu"
            )(x)
x = layers.Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding="same")(x)
x = layers.Flatten(name="x_params")(x)
decoded = tfp.layers.IndependentBernoulli((28, 28, 1), name="x_layer")(x)
vae = keras.Model(input_img, decoded)
vae.compile(optimizer='adam', loss=lambda x, y: -y.log_prob(x))
vae.summary()

Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_38 (Conv2D)          (None, 13, 13, 32)        320       
                                                                 
 conv2d_39 (Conv2D)          (None, 6, 6, 64)          18496     
                                                                 
 flatten_14 (Flatten)        (None, 2304)              0         
                                                                 
 z_params (Dense)            (None, 16)                36880     
                                                                 
 z_layer (IndependentNormal)  ((None, 8),              0         
                              (None, 8))                         
                                                          

In [11]:
from stacked_mnist import StackedMNISTData, DataMode
import numpy as np

generator = StackedMNISTData(mode=DataMode.MONO_BINARY_MISSING, default_batch_size=2048)

x_train, y_train = generator.get_full_data_set(training=True)
x_test, y_test = generator.get_full_data_set(training=False)

# "Translate": Only look at "red" channel; only use the last digit. Use one-hot for labels during training
x_train = x_train[:, :, :, [0]]
y_train = keras.utils.to_categorical((y_train % 10).astype(np.int), 10)
x_test = x_test[:, :, :, [0]]
y_test = keras.utils.to_categorical((y_test % 10).astype(np.int), 10)

/var/folders/w2/4g3c4yrn38g2nwwdhjdlx6fc0000gn/T/ipykernel_4163/4188248072.py:11: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = keras.utils.to_categorical((y_train % 10).astype(np.int), 10)
/var/folders/w2/4g3c4yrn38g2nwwdhjdlx6fc0000gn/T/ipykernel_4163/4188248072.py:13: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precisi

In [67]:
vae.fit(x_train, x_train,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/10
469/469 [==============================] - 33s 66ms/step - loss: 200.3154 - val_loss: 159.0822
Epoch 2/10
469/469 [==============================] - 29s 61ms/step - loss: 151.8313 - val_loss: 146.2973
Epoch 3/10
469/469 [==============================] - 31s 67ms/step - loss: 145.5946 - val_loss: 142.6749
Epoch 4/10
469/469 [==============================] - 29s 61ms/step - loss: 142.7489 - val_loss: 140.7639
Epoch 5/10
469/469 [==============================] - 28s 60ms/step - loss: 141.0995 - val_loss: 139.4569
Epoch 6/10
469/469 [==============================] - 28s 61ms/step - loss: 139.8203 - val_loss: 138.3770
Epoch 7/10
469/469 [==============================] - 28s 60ms/step - loss: 138.9599 - val_loss: 137.7299
Epoch 8/10
469/469 [==============================] - 30s 64ms/step - loss: 138.1588 - val_loss: 136.9034
Epoch 9/10
469/469 [==============================] - 31s 66ms/step - loss: 137.5901 - val_loss: 136.6725
Epoch 10/10
469/469 [=========================

In [ ]:
generator_full = StackedMNISTData(mode=DataMode.MONO_BINARY_COMPLETE, default_batch_size=2048)
x_test, y_test = generator.get_full_data_set(training=False)

# "Translate": Only look at "red" channel; only use the last digit. Use one-hot for labels during training
x_train = x_train[:, :, :, [0]]
x_test = x_test[:, :, :, [0]]

In [ ]:
reconstructed = vae(x_test).mode()
bce = keras.losses.BinaryCrossentropy()
losses = []
for i in range(x_test.shape[0]):
    losses.append(bce(x_test[i], reconstructed[i]).numpy())
losses = np.array(losses)
ind = np.argpartition(losses, -10)[-10:] # Get the indices of the 10 largest losses

In [ ]:
import matplotlib.pyplot as plt
n = 10
plt.figure(figsize=(20, 4))
for i in range(1, n + 1):
    # Display original
    ax = plt.subplot(2, n, i)
    plt.imshow(x_test[ind[i-1]].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()